In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import plot_importance

In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
path = os.getcwd()

In [4]:
os.listdir(path)

['.ipynb_checkpoints',
 '.RData',
 '.Rhistory',
 '.Rproj.user',
 'After_COVID_BL.csv',
 'After_COVID_LIGA.csv',
 'After_COVID_PL.csv',
 'After_COVID_plot',
 'After_COVID_SE.csv',
 'attack numbers.png',
 'baseline_models.ipynb',
 'bases3.RData',
 'bayesian_model.R',
 'Before_COVID_BL.csv',
 'Before_COVID_LIGA.csv',
 'Before_COVID_PL.csv',
 'Before_COVID_SE.csv',
 'BL.csv',
 'BL.RData',
 'BL_fixed_samples.csv',
 'BL_indep_samples.csv',
 'BL_sample.RData',
 'BL_sampling_fixed.RData',
 'BL_sampling_indep.RData',
 'BL_train_sample.RData',
 'data',
 'data_fixed.rdata',
 'data_indep.rdata',
 'experiments.R',
 'full___.RData',
 'home_win_before.csv',
 'LIGA.csv',
 'LIGA.RData',
 'LIGA_fixed_samples.csv',
 'LIGA_indep_samples.csv',
 'LIGA_sample.RData',
 'LIGA_sampling_fixed.RData',
 'LIGA_sampling_indep.RData',
 'LIGA_train_sample.RData',
 'ms3.Rdata',
 'note.R',
 'PL.csv',
 'PL.RData',
 'plot',
 'plot 그리기.R',
 'plot.R',
 'PL_fixed_samples.csv',
 'PL_indep_samples.csv',
 'PL_sampling.RData',
 

In [5]:
########### read csv files

In [6]:
PL = pd.read_csv('PL.csv').iloc[:,1:]
BL = pd.read_csv('BL.csv').iloc[:,1:]
LIGA = pd.read_csv('LIGA.csv').iloc[:,1:]
SE = pd.read_csv('SE.csv').iloc[:,1:]
df = pd.concat([PL,LIGA,BL,SE], axis=0)

In [7]:
df = df.dropna()

In [56]:
def win(x):
    if x > 0:
        return 1
    if x == 0:
        return 0
    else :
        return -1

In [57]:
result = list(map(win, (df['full_home_score'] - df['full_away_score'])))
len(result)

12144

In [58]:
df['result'] = result

In [59]:
df[df['result']==0].shape[0]/11250

0.26622222222222225

In [60]:
result[result == 0]

0

In [61]:
df[df['result']>0].shape[0]/11250

0.49217777777777777

In [62]:
df[df['result']<0].shape[0]/11250

0.32106666666666667

In [63]:
def null_classifier(n_games):
    random_list = np.random.rand(n_games)
    def classifier(x):
        if x < 0.460:
            return 1
        elif x < 0.705:
            return 0
        else :
            return -1
    return(np.array(list(map(classifier, random_list))))    

In [64]:
aa = null_classifier(12144)

In [65]:
aa[aa == df['result']].shape[0]/12144

0.35778985507246375

In [66]:
df.columns

Index(['home_shot', 'away_shot', 'home_possession', 'away_possession',
       'home_pass_success', 'away_pass_success', 'home_dribbles',
       'away_dribbles', 'home_aerials_won', 'away_aerials_won', 'home_tackles',
       'away_tackles', 'home_corners', 'away_corners', 'home_dispossessed',
       'away_dispossessed', 'home_missing_player', 'away_missing_player',
       'home', 'away', 'half_home_score', 'half_away_score', 'full_home_score',
       'full_away_score', 'kick_off', 'day', 'date', 'matchup_home_goals',
       'matchup_away_goals', 'matchup_home_wins', 'matchup_draw',
       'matchup_away_wins', 'home_total_att', 'away_total_att',
       'home_open_att', 'away_open_att', 'home_set_att', 'away_set_att',
       'home_counter_att', 'away_counter_att', 'home_pk_att', 'away_pk_att',
       'home_own_att', 'away_own_att', 'home_total_passes',
       'away_total_passes', 'home_crosses_passes', 'away_crosses_passes',
       'home_long_balls', 'away_long_balls', 'home_short_passes'

In [67]:
df

,home_shot,away_shot,home_possession,away_possession,home_pass_success,away_pass_success,home_dribbles,away_dribbles,home_aerials_won,away_aerials_won,...,home_last_5_conceded,away_last_5_conceded,home_last_5_passes,away_last_5_passes,home_last_5_shot,away_last_5_shot,season,home_uefa,away_uefa,result
0,14,5,45.1,54.9,79,80,11,5,28,33,...,12.0,3.0,2315.0,2141.0,59.0,53.0,PL1920,17.092,17.092,0
1,20,3,67.1,32.9,92,84,13,3,6,2,...,4.0,4.0,3789.0,2730.0,82.0,48.0,PL1920,123.092,118.092,1
2,10,12,38.8,61.2,75,83,10,14,14,19,...,8.0,1.0,2507.0,2433.0,57.0,72.0,PL1920,95.092,95.092,0
3,9,22,55.0,45.0,71,69,10,18,13,23,...,5.0,10.0,2420.0,2135.0,53.0,71.0,PL1920,17.092,24.592,-1
4,8,15,31.7,68.3,63,81,5,5,22,21,...,8.0,4.0,1951.0,2418.0,47.0,67.0,PL1920,17.092,39.092,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,14,9,43.7,56.3,82,84,6,11,12,4,...,6.0,7.0,2232.0,1734.0,84.0,66.0,SE1920,14.945,53.945,0
3692,13,9,53.3,46.7,78,79,10,12,22,13,...,5.0,14.0,1904.0,2189.0,61.0,79.0,SE1920,14.945,14.945,-1
3693,13,8,48.0,52.0,80,79,5,11,22,21,...,15.0,2.0,2051.0,2271.0,80.0,57.0,SE1920,15.945,14.945,1
3694,17,13,53.9,46.1,91,86,10,7,7,4,...,5.0,6.0,3117.0,2672.0,85.0,86.0,SE1920,138.945,45.945,1


In [68]:
date_no = df['date_no']

In [69]:
df = df[['day','home_missing_player','away_missing_player','matchup_home_wins','matchup_draw',
    'matchup_away_wins', 'home_last_5_points', 'away_last_5_points', 'home_last_5_goals',
   'away_last_5_goals', 'home_last_5_conceded','away_last_5_conceded','home_last_5_passes',
   'away_last_5_passes', 'home_last_5_shot', 'away_last_5_shot','home_uefa','away_uefa','result']]

In [70]:
df['day'] = ['weekend' if i in ['Sat', 'Sun'] else 'week' for i in df['day']]

<ipython-input-70-bec4574b6918>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = ['weekend' if i in ['Sat', 'Sun'] else 'week' for i in df['day']]


In [71]:
df['result'] = df['result']+1

<ipython-input-71-79a67650dfaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'] = df['result']+1


In [72]:
df.head()

,day,home_missing_player,away_missing_player,matchup_home_wins,matchup_draw,matchup_away_wins,home_last_5_points,away_last_5_points,home_last_5_goals,away_last_5_goals,home_last_5_conceded,away_last_5_conceded,home_last_5_passes,away_last_5_passes,home_last_5_shot,away_last_5_shot,home_uefa,away_uefa,result
0,week,2,2,17,33,50,3.0,10.0,5.0,5.0,12.0,3.0,2315.0,2141.0,59.0,53.0,17.092,17.092,1
1,week,0,3,100,0,0,9.0,11.0,4.0,10.0,4.0,4.0,3789.0,2730.0,82.0,48.0,123.092,118.092,2
2,week,5,1,33,0,67,7.0,11.0,9.0,8.0,8.0,1.0,2507.0,2433.0,57.0,72.0,95.092,95.092,1
3,week,2,1,17,17,67,4.0,3.0,1.0,4.0,5.0,10.0,2420.0,2135.0,53.0,71.0,17.092,24.592,0
4,weekend,2,4,33,0,67,4.0,5.0,6.0,6.0,8.0,4.0,1951.0,2418.0,47.0,67.0,17.092,39.092,1


In [73]:
df[['week','weekend']] = pd.get_dummies(df['day'])

C:\Users\jaemini_man\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [74]:
df = df.drop(['day'], axis=1)

In [75]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(df.values)

In [76]:
date_no

0       18430
1       18430
2       18432
3       18432
4       18433
        ...  
3691    18329
3692    18329
3693    18329
3694    18329
3695    18330
Name: date_no, Length: 12144, dtype: int64

In [77]:
df = pd.DataFrame(x_scaled)

In [78]:
df[17] = df[17].astype(np.str)

In [79]:
X = df.drop([17],axis=1)

In [80]:
y = np.array(result)

In [81]:
np.shape(X)

(12144, 19)

In [128]:
X_test = X[np.array(date_no)>=18628]
y_test = y[np.array(date_no)>=18628]
X_train = X[np.array(date_no)<18628]
y_train = y[np.array(date_no)<18628]

In [167]:
sum(y_train==1)/len(y_train)

0.4571527000258376

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

In [129]:
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear', C=0.1)
lm.fit(X_train, y_train)

LogisticRegression(C=0.1, multi_class='ovr', solver='liblinear')

In [130]:
lm

LogisticRegression(C=0.1, multi_class='ovr', solver='liblinear')

In [131]:
lm.score(X_test, y_test)

0.5347091932457786

In [146]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(4, 4), random_state=1, max_iter = 1000)

In [147]:
clf.fit(np.array(X_train), np.array(y_train))

C:\Users\jaemini_man\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(4, 4), max_iter=1000,
              random_state=1, solver='lbfgs')

In [148]:
clf.predict(X_test)

array([ 1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1,  1, -1,
        1,  1,  1, -1,  1,  1,  1, -1,  1, -1,  1,  1,  1, -1, -1, -1, -1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1,  1,
        1,  1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1, -1,
        1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1, -1, -1,  1, -1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1,  1, -1, -1, -1,  1,  1,
        1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1,
        1,  1, -1, -1,  1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  1,  1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1,
       -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
       -1, -1, -1,  1,  1

In [149]:
clf.score(X_test,y_test)

0.5384615384615384

In [150]:
clf.predict_proba(X_test)

array([[0.13340649, 0.25704883, 0.60954468],
       [0.25247579, 0.31447229, 0.43305192],
       [0.24316801, 0.22811467, 0.52871732],
       ...,
       [0.46638075, 0.25373011, 0.27988915],
       [0.27178741, 0.23874793, 0.48946466],
       [0.48501333, 0.27193036, 0.24305631]])

In [151]:
np.array(X_train)

array([[0.13333333, 0.125     , 0.17      , ..., 0.0297311 , 1.        ,
        0.        ],
       [0.        , 0.1875    , 1.        , ..., 0.61898567, 1.        ,
        0.        ],
       [0.33333333, 0.0625    , 0.33      , ..., 0.48479898, 1.        ,
        0.        ],
       ...,
       [0.33333333, 0.1875    , 0.5       , ..., 0.01720507, 0.        ,
        1.        ],
       [0.06666667, 0.1875    , 0.67      , ..., 0.19806538, 0.        ,
        1.        ],
       [0.13333333, 0.125     , 0.33      , ..., 0.01720507, 1.        ,
        0.        ]])

In [152]:
np.array(y_train)

array([0, 1, 0, ..., 1, 1, 1])

In [153]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19
0,0.133333,0.1250,0.17,0.33,0.50,0.200000,0.666667,0.20,0.20,0.571429,0.142857,0.396567,0.358468,0.401869,0.333333,0.029731,0.029731,1.0,0.0
1,0.000000,0.1875,1.00,0.00,0.00,0.600000,0.733333,0.16,0.40,0.190476,0.190476,0.818312,0.531959,0.616822,0.288288,0.648157,0.618986,1.0,0.0
2,0.333333,0.0625,0.33,0.00,0.67,0.466667,0.733333,0.36,0.32,0.380952,0.047619,0.451502,0.444477,0.383178,0.504505,0.484799,0.484799,1.0,0.0
3,0.133333,0.0625,0.17,0.17,0.67,0.266667,0.200000,0.04,0.16,0.238095,0.476190,0.426609,0.356701,0.345794,0.495495,0.029731,0.073488,1.0,0.0
4,0.133333,0.2500,0.33,0.00,0.67,0.266667,0.333333,0.24,0.24,0.380952,0.190476,0.292418,0.440059,0.289720,0.459459,0.029731,0.158084,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,0.066667,0.1875,0.00,0.17,0.83,0.200000,0.333333,0.08,0.28,0.285714,0.333333,0.372818,0.238586,0.635514,0.450450,0.017205,0.244739,0.0,1.0
12140,0.466667,0.1875,0.00,0.00,1.00,0.466667,0.000000,0.24,0.20,0.238095,0.666667,0.278970,0.372607,0.420561,0.567568,0.017205,0.017205,0.0,1.0
12141,0.333333,0.1875,0.50,0.17,0.33,0.266667,0.600000,0.28,0.24,0.714286,0.095238,0.321030,0.396760,0.598131,0.369369,0.023039,0.017205,0.0,1.0
12142,0.066667,0.1875,0.67,0.33,0.00,0.600000,0.533333,0.36,0.36,0.238095,0.285714,0.626037,0.514875,0.644860,0.630631,0.740646,0.198065,0.0,1.0


In [154]:
scaler = StandardScaler()

In [155]:
scaler.fit(X_train.values)

StandardScaler()

In [156]:
X_train = scaler.transform(X_train.values)
X_test = scaler.transform(X_test.values)

In [157]:
rf = RandomForestClassifier(n_estimators=300, max_features=2)
rf.fit(X_train, y_train)

RandomForestClassifier(max_features=2, n_estimators=300)

In [158]:
y_train+1

array([1, 2, 1, ..., 2, 2, 2])

In [159]:
rf.predict(X_test)

array([ 1,  1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1,  1,  1,  1,  1, -1,
        0,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1,  1, -1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1,  1,
        1,  1, -1, -1, -1,  1,  0,  1, -1, -1,  1,  1, -1,  1,  1,  1, -1,
        1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1, -1,  1, -1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0,  1, -1,  1,  1,
        1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,
        1,  1, -1,  1,  1,  1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,
        1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,  1, -1, -1,  0,  1,  0,
        1,  0,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1,  1,
       -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
       -1, -1, -1,  1,  1

In [160]:
rf.score(X_test,y_test)

0.5403377110694184

In [161]:
scores = rf.feature_importances_

In [162]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19
0,0.133333,0.1250,0.17,0.33,0.50,0.200000,0.666667,0.20,0.20,0.571429,0.142857,0.396567,0.358468,0.401869,0.333333,0.029731,0.029731,1.0,0.0
1,0.000000,0.1875,1.00,0.00,0.00,0.600000,0.733333,0.16,0.40,0.190476,0.190476,0.818312,0.531959,0.616822,0.288288,0.648157,0.618986,1.0,0.0
2,0.333333,0.0625,0.33,0.00,0.67,0.466667,0.733333,0.36,0.32,0.380952,0.047619,0.451502,0.444477,0.383178,0.504505,0.484799,0.484799,1.0,0.0
3,0.133333,0.0625,0.17,0.17,0.67,0.266667,0.200000,0.04,0.16,0.238095,0.476190,0.426609,0.356701,0.345794,0.495495,0.029731,0.073488,1.0,0.0
4,0.133333,0.2500,0.33,0.00,0.67,0.266667,0.333333,0.24,0.24,0.380952,0.190476,0.292418,0.440059,0.289720,0.459459,0.029731,0.158084,0.0,1.0


In [163]:
scores

array([0.0473986 , 0.04760634, 0.03757705, 0.03215182, 0.03976806,
       0.05172301, 0.05150773, 0.05177823, 0.05278748, 0.05154008,
       0.05135677, 0.07955497, 0.08225134, 0.0699485 , 0.07098633,
       0.08192445, 0.08145205, 0.0093881 , 0.00929911])

In [58]:
['day','home_missing_player','away_missing_player','matchup_home_wins','matchup_draw',
    'matchup_away_wins', 'home_last_5_points', 'away_last_5_points', 'home_last_5_goals',
   'away_last_5_goals', 'home_last_5_conceded','away_last_5_conceded','home_last_5_passes',
   'away_last_5_passes', 'home_last_5_shot', 'away_last_5_shot','home_uefa','away_uefa','result']

RangeIndex(start=0, stop=20, step=1)

In [45]:
(pd.get_dummies(y_train)).shape

(8332, 3)

In [46]:
dtrain = xgb.DMatrix(data=X_train, label = y_train+1)
dtest = xgb.DMatrix(data=X_test, label = y_test+1)

In [590]:
params = {'n_estimators' : 200,
          'subsample' : 0.5,
          'colsample_bytree' : 0.5,
          'num_class' : 3,
         'objective' : 'multi:softprob',
         'eval_metric' : 'mlogloss',
         'early_stoppings' : 100 }

num_rounds = 10

In [591]:
wlist = [(dtrain, 'train'), (dtest,'eval')]
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)

[15:35:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stoppings, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:1.04042	eval-mlogloss:1.05583
[1]	train-mlogloss:1.00311	eval-mlogloss:1.03384
[2]	train-mlogloss:0.97396	eval-mlogloss:1.01851
[3]	train-mlogloss:0.95132	eval-mlogloss:1.01088
[4]	train-mlogloss:0.93511	eval-mlogloss:1.00512
[5]	train-mlogloss:0.92320	eval-mlogloss:1.00240
[6]	train-mlogloss:0.91204	eval-mlogloss:1.00112
[7]	train-mlogloss:0.90179	eval-mlogloss:1.00011
[8]	train-mlogloss:0.89257	eval-mlogloss:1.00035
[9]	train-mlogloss:0.88341	eval-mlogloss:0.99960


In [592]:
pred_probs = xgb_model.predict(dtest)

In [593]:
y_pred = np.argmax(pred_probs, axis=1)

In [594]:
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))

In [595]:
get_clf_eval(y_test+1,y_pred)

오차행렬:
 [[ 448   25  509]
 [ 200   30  611]
 [ 236   39 1277]]

정확도: 0.5200
